In [19]:
import os
from datetime import datetime, timedelta
import config as cfg
import logging
import pandas as pd
from typing import List
from data import save_data
from pipeline import retrieve_data_in_range, validate_and_save_data, transform_to_ts_data
import hopsworks
import config as config

In [20]:
# Fetch data from the API for the last 28 days
current_date = pd.to_datetime(datetime.utcnow()).floor('H')
end = current_date - timedelta(days=7*10)
start = end - timedelta(days=7*52)

# Define the base URL for the API
base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/"

start, end

(Timestamp('2023-05-27 22:00:00'), Timestamp('2024-05-25 22:00:00'))

In [21]:
# STREAM_DATA_DIR_RAW = RAW_DATA_DIR / "2023-2024"
# STREAM_DATA_DIR_BRONZE = BRONZE_DATA_DIR / "2023-2024"
# STREAM_DATA_DIR_SILVER = SILVER_DATA_DIR / "2023-2024"
# STREAM_DATA_DIR_GOLD = GOLD_DATA_DIR / "2023-2024"
# INPUT_RAW_DIR = RAW_DATA_DIR / "2023-2024"/f'yellow_tripdata_{start.year}-{start.month:02d}_to_{end.year}-{end.month:02d}.parquet'
# INPUT_SILVER_DIR = STREAM_DATA_DIR_SILVER /f'ts_data_{start.year}-{start.month:02d}_to_{end.year}-{end.month:02d}.parquet'

In [22]:
# Example usage
base_path = "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/raw"

df = retrieve_data_in_range(start, end, RAW_DATA_DIR)
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-06-01 00:08:48,2023-06-01 00:29:41,1.0,3.40,1.0,N,140,238,1,21.90,3.50,0.5,6.70,0.0,1.0,33.60,2.5,0.00
1,1,2023-06-01 00:15:04,2023-06-01 00:25:18,0.0,3.40,1.0,N,50,151,1,15.60,3.50,0.5,3.00,0.0,1.0,23.60,2.5,0.00
2,1,2023-06-01 00:48:24,2023-06-01 01:07:07,1.0,10.20,1.0,N,138,97,1,40.80,7.75,0.5,10.00,0.0,1.0,60.05,0.0,1.75
3,2,2023-06-01 00:54:03,2023-06-01 01:17:29,3.0,9.83,1.0,N,100,244,1,39.40,1.00,0.5,8.88,0.0,1.0,53.28,2.5,0.00
4,2,2023-06-01 00:18:44,2023-06-01 00:27:18,1.0,1.17,1.0,N,137,234,1,9.30,1.00,0.5,0.72,0.0,1.0,15.02,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38916735,2,2024-05-31 23:45:00,2024-06-01 00:09:00,NaN,4.07,NaN,None,88,186,0,29.93,0.00,0.5,0.00,0.0,1.0,33.93,NaN,NaN
38916736,2,2024-05-31 23:10:52,2024-05-31 23:38:05,NaN,4.81,NaN,None,231,143,0,30.71,0.00,0.5,0.00,0.0,1.0,34.71,NaN,NaN
38916737,2,2024-05-31 23:23:57,2024-05-31 23:38:35,NaN,2.03,NaN,None,142,140,0,17.15,0.00,0.5,4.23,0.0,1.0,25.38,NaN,NaN
38916738,2,2024-05-31 23:44:12,2024-05-31 23:47:44,NaN,0.94,NaN,None,141,263,0,-1.00,0.00,0.5,0.00,0.0,1.0,3.00,NaN,NaN


In [23]:
save_data(df, STREAM_DATA_DIR_RAW, f'yellow_tripdata_{start.year}-{start.month:02d}_to_{end.year}-{end.month:02d}.parquet')

Saving data: 100%|██████████| 38916740/38916740 [00:10<00:00, 3661779.05rows/s]

Data saved to "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/raw/2023-2024/yellow_tripdata_2023-05_to_2024-05.parquet"


In [24]:
validated = validate_and_save_data(STREAM_DATA_DIR_RAW, start, end)

save_data(validated, STREAM_DATA_DIR_BRONZE, f'validated_yellow_tripdata_{start.year}-{start.month:02d}_to_{end.year}-{end.month:02d}.parquet')

Saving data: 100%|██████████| 37298630/37298630 [00:02<00:00, 13867115.40rows/s]

Data saved to "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/bronze/2023-2024/validated_yellow_tripdata_2023-05_to_2024-05.parquet"


In [25]:
validated = pd.read_parquet(f'{STREAM_DATA_DIR_BRONZE}/validated_yellow_tripdata_{start.year}-{start.month:02d}_to_{end.year}-{end.month:02d}.parquet')

transformed = transform_to_ts_data(validated)

In [26]:
save_data(transformed, STREAM_DATA_DIR_SILVER, f'ts_data_{start.year}-{start.month:02d}_to_{end.year}-{end.month:02d}.parquet')

Saving data: 100%|██████████| 2505864/2505864 [00:00<00:00, 17657132.92rows/s]

Data saved to "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/silver/2023-2024/ts_data_2023-05_to_2024-05.parquet"


In [27]:
ts_data = pd.read_parquet(INPUT_SILVER_DIR)

# string to datetime
ts_data['pickup_time'] = pd.to_datetime(ts_data['pickup_time'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_time'].astype(int) // 10**6

# Convert the pickup_location_id column to bigint
ts_data['pickup_location_transformed'] = ts_data['pickup_location_id'].astype('int64')

In [28]:
# Connect to the project
project = hopsworks.login(
    project=cfg.HOPSWORKS_PROJECT_NAME,
    api_key_value=cfg.HOPSWORKS_API_KEY
)

# Connect to the feature store
feature_store = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/933012
Connected. Call `.close()` to terminate connection gracefully.


In [29]:
# Connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=cfg.FEATURE_GROUP_NAME,
    version=cfg.FEATURE_GROUP_VERSION,
    description="ts data hourly frequency",
    primary_key = ['pickup_location_transformed', 'pickup_ts'],
    event_time='pickup_ts',
)

In [30]:
# Insert features into the feature group
feature_group.insert(ts_data, write_options={"wait_for_job": False},
                        overwrite=True)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/933012/fs/926787/fg/1069387


Uploading Dataframe: 0.00% |          | Rows 0/2505864 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: hourly_features_2023_2024_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/933012/jobs/named/hourly_features_2023_2024_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x30b4fe410>, None)